In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import pprint
from boto.s3.connection import S3Connection
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
import pylab as pl
import numpy as np
import pandas as pd

In [36]:
def convert(x):
    ''' Convert a json string to a flat python dictionary
    which can be passed into Pandas. '''
    ob = json.loads(x)
    for k, v in ob.items():
        if isinstance(v, list):
            ob[k] = ','.join(v)
        elif isinstance(v, dict):
            for kk, vv in v.items():
                ob['%s_%s' % (k, kk)] = vv
            del ob[k]
    return ob

def convert_2(x):
    ''' Convert a json string to a flat python dictionary
    which can be passed into Pandas. '''
    ob = json.loads(x)
    return {"business_id":ob["business_id"], "stars":ob["stars"], "date":ob["date"]}

# Returning Business DataFrame
json_filename = 'yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json'
df_business = pd.DataFrame([convert(line) for line in file(json_filename)])

json_filename = 'yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json'
df_review = pd.DataFrame([convert_2(line) for line in file(json_filename)])

In [96]:
# Find the Las Vegas Restaurants
#df_business_1 = df_business[["business_id","name", "city", "stars"]][df_business['city']=="Las Vegas"]

# Merge the restaurants and businesses together
#df_review_business = pd.merge(df_business_1, df_review, left_on='business_id', right_on='business_id', how='inner')

# Find the Vegas Restaurants opened before 2011
df_review_business.to_csv("business_reviews.csv")

In [101]:
d = {}
for line in file("business_reviews.csv"):
    ele = line.split(",")
    if ele[1] not in d:
        d[ele[1]] = ele[5]
    else:
        if ele[5] < d[ele[1]]:
            d[ele[1]] = ele[5]

In [108]:
f = open("business_reviews_final.csv", "w")
newcount = 0
for line in file("business_reviews.csv"):
    ele = line.split(",")
    ty = "old"
    if d[ele[1]] >= '2011-01-01':
        ty = "new"
        newcount+=1
    # Not considering new businesses after 2012
    if d[ele[1]] <= '2012-01-01':
        f.write(line+","+ty+"\n")
f.close()